In [1]:
import os
import json
import pickle
import cv2
import numpy as np
from tqdm.notebook import tqdm

## Helper Functions

In [2]:
# a function for reading Json files
def json_reader(dictionary, jsn, frame):
    '''
    The function takes a dictionary and a string mentioning
    the name of the json file, and a frame index, and it
    reads index-related info from json files and record them
    in the dictionary.
    '''
    # set the path to the json files
    path = 'Data_MIT/' + frame[:5] + '/'
    # get the index of the frame
    idx = int(frame[5:])
    # read the face json file
    with open(str(path + 'apple' + jsn + '.json'), 'r') as file:
        jsn_info = json.load(file)    
    # add the face patch coordinates to the dictionary
    dictionary[jsn] = [round(jsn_info['X'][idx]),
                       round(jsn_info['Y'][idx]),
                       round(jsn_info['W'][idx]),
                       round(jsn_info['H'][idx])]

    
# a function to cut, resize, and save a patch
def patch_reader(image, coordinates, name):
    '''
    The function takes an image and a list of coordinates (int),
    and a name (string), and cuts a patch out of the image,
    resizes it, and saves it under the given name.
    '''
    # check if the patches falls out of the frame
    # -------------------------------------------
    # whether the X coordinate is negative
    if coordinates[0] < 0:
        # if so, fill the shortage by replicating the edge 
        image = cv2.copyMakeBorder(image, 0, 0, np.abs(coordinates[0]), 0, cv2.BORDER_REPLICATE)
        # and reset the X value regarding the new frame
        coordinates[0] = 0

    # whether Y is negative
    if coordinates[1] < 0:
        # if so, fill the shortage by replicating the edge 
        image = cv2.copyMakeBorder(image, np.abs(coordinates[1]), 0, 0, 0, cv2.BORDER_REPLICATE)
        # and reset the Y value regarding the new frame
        coordinates[1] = 0

    # whether X+W is larger than the frame width
    if coordinates[0]+coordinates[2] > image.shape[1]:
        # if so, fill the shortage (diff) by replicating the edge
        diff = np.abs(image.shape[1]-(coordinates[0]+coordinates[2]))
        image = cv2.copyMakeBorder(image, 0, 0, 0, diff, cv2.BORDER_REPLICATE)

    # whether Y+H is larger than the frame height
    if coordinates[1]+coordinates[3] > image.shape[0]:
        # so we fill the shortage (diff) by replicating the edge
        diff = np.abs(image.shape[0]-(coordinates[1]+coordinates[3]))
        image = cv2.copyMakeBorder(image, 0, diff, 0, 0, 0, cv2.BORDER_REPLICATE)

        
    # cut the patch
    patch = image[coordinates[1]:coordinates[1]+coordinates[3],
                  coordinates[0]:coordinates[0]+coordinates[2]]
    
    # resize the patch
    patch_res = cv2.resize(patch, (224, 224))
    
    # set the path and name for the image
    path = 'Data_MTCNN/Patches/' + name + '.jpg'
    # save the image
    cv2.imwrite(path, patch_res);
    

## Patch Creator

In [3]:
# read the pkl file and load the data
with open('Data_MTCNN/frames.pkl','rb') as f:
    frames = pickle.load(f)
frames.sort()

In [4]:
# set counter for naming the patches
counter = 0

# iterate over frames
for frame in tqdm(frames):
    # define the dictionary for recording the info
    coords = {}
    # list of name of json files
    json_list = ['Face', 'LeftEye', 'RightEye']
    
    # iterate over all json files and read/record the info
    for jsn in json_list:
        # read each json file
        json_reader(coords, jsn, frame)
    
    # adjust the eyes' coordinates to express the location
    # for within the frame
    coords['LeftEye'][0] += coords['Face'][0]
    coords['LeftEye'][1] += coords['Face'][1]
    coords['RightEye'][0] += coords['Face'][0]
    coords['RightEye'][1] += coords['Face'][1]
    
    # set the image name
    name = '0'*(5-len(str(counter))) + str(counter)
    # set the path to the image
    path = 'Data_MIT/' + frame[:5] + '/frames/' + '0'*(5-len(frame[5:])) + frame[5:] + '.jpg'
    # read the image
    img = cv2.imread(path)
    
    # iterate over two eyes
    for patch in ['Face', 'LeftEye', 'RightEye']:
        # save the patch
        patch_reader(img, coords[patch], name + '_' + patch[0].lower())
        
    # count
    counter += 1
    